In [ ]:
!pip install accelerate
!pip install rouge_score
import nltk
nltk.download('popular')

In [ ]:
import os

if os.getcwd() == '/content/NLP-Business-News-Summarization':
  %cd ..
  !rm -r ./NLP-Business-News-Summarization/

# Remove Colab default sample_data
if os.path.exists('./sample_data'):
  !rm -r ./sample_data

repo_name = "NLP-Business-News-Summarization" # Repository Name

if not os.path.isdir('./' + repo_name):
  # Clone GitHub Files
  git_path = 'https://github.com/christianwhollar/NLP-Business-News-Summarization.git'
  !git clone "{git_path}"

# Change to new directory
if not os.path.basename(os.getcwd()) == repo_name:
  %cd "{repo_name}"
  
  # Install dependencies from requirements.txt file
  !pip install -r requirements.txt
  !pip install transformers[torch]
  !python -c "from accelerate.utils import write_basic_config; write_basic_config(mixed_precision='fp16')"

In [ ]:
from google.colab import files

if 'kaggle.json' not in os.listdir():
  files.upload()
  ! mkdir ~/.kaggle
  ! cp kaggle.json /content/NLP-Business-News-Summarization/data/
  ! chmod 600 /content/NLP-Business-News-Summarization/data/kaggle.json
  ! cp kaggle.json ~/.kaggle/
  ! chmod 600 ~/.kaggle/kaggle.json

In [1]:
%run scripts/setup.py

Deleted directory: data/processed/test
Deleted directory: data/processed/train
Deleted directory: data/processed/val
Beginning download...
Starting pariza/bbc-news-summary Download
Completed pariza/bbc-news-summary Download
Starting rmisra/news-category-dataset Download
Completed rmisra/news-category-dataset Download
Download completed!
json converted to DataFrame!
Articles downloaded!
Moving and renaming files...
Shutil task completed!


Map:   0%|          | 0/12 [00:00<?, ? examples/s]

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

c:\Users\19105\anaconda3\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/12 [00:00<?, ?it/s]

...

{'eval_loss': 2.830634355545044, 'eval_rouge1': 0.2177, 'eval_rouge2': 0.0762, 'eval_rougeL': 0.1688, 'eval_rougeLsum': 0.1688, 'eval_gen_len': 19.0, 'eval_runtime': 4.2332, 'eval_samples_per_second': 0.709, 'eval_steps_per_second': 0.236, 'epoch': 3.0}


  0%|          | 0/1 [00:00<?, ?it/s]

{'eval_loss': 2.8091440200805664, 'eval_rouge1': 0.2177, 'eval_rouge2': 0.0762, 'eval_rougeL': 0.1688, 'eval_rougeLsum': 0.1688, 'eval_gen_len': 19.0, 'eval_runtime': 4.3682, 'eval_samples_per_second': 0.687, 'eval_steps_per_second': 0.229, 'epoch': 4.0}
{'train_runtime': 92.8055, 'train_samples_per_second': 0.517, 'train_steps_per_second': 0.129, 'train_loss': 2.333381017049154, 'epoch': 4.0}


  0%|          | 0/1 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (599 > 512). Running this sequence through the model will result in indexing errors
